In [1]:
import os
from langchain.llms import HuggingFaceHub
from secret_key import huggingface_api_key

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_api_key

In [3]:
model_id = "google/flan-t5-xxl"

llm = HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature":0.2, "max_length":64})

In [4]:
prompt_template_name = PromptTemplate(input_variables=["cusine"],
                                      template="I want to open a {cusine} resturant, suggest a fancy name for this"
                                      )

In [5]:
chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("American")

'The Blues Brothers'